# Machine Learning Project: French Grand Debat

### Project Introduction :

First, we would like to choose a problematic according to the data we collected. The aim of this project could be to answer the following questions:

* What are the 5 most important ideas on each theme?
* Can we build different profiles of people with their ideas on the 4 themes?
* Can we predict themes about which people could be interested in if those people answered questions only about 3 on 4 themes? 
 
#### Doability :

We are going to import the data, then check if the ids of people who submitted ideas on different themes are the same. If not, we won't be able to answer the second question.


In [2]:
# librairies imported
import src.utils as ut
import numpy as np
import pandas as pd
import string
from src.kmeans_embeddings import FeaturesExtractor
from sklearn.cluster import KMeans
from src.utils import (read_data, get_open_reponses, get_ids_open_reponses)
from sklearn.mixture import GaussianMixture
import multiprocessing

#### Data extraction

In [5]:
df_fiscalite = ut.read_data('data/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.json')
df_democratie = ut.read_data('data/DEMOCRATIE_ET_CITOYENNETE.json')
df_ecologie = ut.read_data('data/LA_TRANSITION_ECOLOGIQUE.json')
df_organisation = ut.read_data('data/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.json')

dfs = np.array([["fiscalite", df_fiscalite], ["democratie", df_democratie], ["ecologie", df_ecologie], ["organisation", df_organisation]])

# open responses
df_resp_fis = get_open_reponses(df_fiscalite)
df_resp_dem = get_open_reponses(df_democratie)
df_resp_eco = get_open_reponses(df_ecologie)
df_resp_org = get_open_reponses(df_organisation)

dfs_responses = np.array([["responses fiscalite", df_resp_fis], ["responses democratie", df_resp_dem], ["responses ecologie", df_resp_eco], ["responses organisation", df_resp_org]])

NameError: name 'get_open_reponses' is not defined

In [3]:
np.random.seed(1)
for i in np.random.randint(len(df_fiscalite), size=5):
    auth = df_fiscalite.loc[i, 'authorId']
    print("Author ID : " + auth)

    dfs = np.array([["fiscalite", df_fiscalite], ["democratie", df_democratie], ["ecologie", df_ecologie], ["organisation", df_organisation]])
    for df in dfs:
        code = df[1].loc[df[1]['authorId'] == auth, 'authorZipCode']
        if(len(code) > 0):
            code = code.values[0]
            print("* In " + df[0] + " survey, author has zip code : " + str(code))
        else:
            print("* In " + df[0] + " survey, author has not answered...")
    print("\n############################\n")

Author ID : VXNlcjoxMjViYWQ4Yi0xZmM0LTExZTktOTRkMi1mYTE2M2VlYjExZTE=
* In fiscalite survey, author has zip code : 93320
* In democratie survey, author has not answered...
* In ecologie survey, author has not answered...
* In organisation survey, author has not answered...

############################

Author ID : VXNlcjplYWEyMzA2MC0xZGEzLTExZTktOTRkMi1mYTE2M2VlYjExZTE=
* In fiscalite survey, author has zip code : 82290
* In democratie survey, author has zip code : 82290
* In ecologie survey, author has not answered...
* In organisation survey, author has zip code : 82290

############################

Author ID : VXNlcjo5ZjllMTFiZS0xYTQ3LTExZTktOTRkMi1mYTE2M2VlYjExZTE=
* In fiscalite survey, author has zip code : 59700
* In democratie survey, author has not answered...
* In ecologie survey, author has not answered...
* In organisation survey, author has not answered...

############################

Author ID : VXNlcjplZjFhMGViMS0xZTU4LTExZTktOTRkMi1mYTE2M2VlYjExZTE=
* In fiscalite su

**From the previous lines, we can see that the `authorId` is likely to be an unique id whaterver the dataframe (zip code is the same).**

### How many people answered open questions of several themes ?

In order to build type-profiles, we need a large number of people who answered open questions on several themes. That is what we will try to find with the following code.

**Be careful, this cell takes a lot of time to run !**

source file : `create_author_tabs.py`

In [18]:
# allAuthIds is the sets of all the authorIds
allAuthIds = []
for i in range(4):
    allAuthIds.extend(set(dfs_responses[i,1]['authorId'].values))
allAuthIds = set(allAuthIds)

# all_auth_id_array is the sorted array of all the authorIds
all_auth_id_array = np.sort(np.array(list(allAuthIds)))

# auth_answers_count[i,j] is 1 if all_auth_id_array[i] has answered survey dfs[j]
auth_answers_count = np.zeros((len(allAuthIds), 4), dtype=int)
for j in range(4):
    for i in range(len(all_auth_id_array)):
        auth = all_auth_id_array[i]
        line = dfs_responses[j,1].loc[dfs_responses[j,1]['authorId'] == auth]
        if(len(line) > 0):
            auth_answers_count[i,j] = auth_answers_count[i,j] + 1

            
# This cell aims to save the auth_answers_count array in auth_answers_count.csv
np.savetxt("auth_answers_count.csv", auth_answers_count, fmt='%1u', delimiter=",")

# list of the authors that answered the four themes
four_surveys_taken_auth_ids = [all_auth_id_array[i] for i in range(len(all_auth_id_array)) if number_of_survey_taken[i] == 4]

np.savetxt("four_surveys_taken_auth_ids.csv", four_surveys_taken_auth_ids, delimiter=",")

KeyboardInterrupt: 

**You can run this cell instead to load auth_answers_count from csv :**

In [6]:
# Read auth_answers_count from auth_answers_count.csv
auth_answers_count = np.loadtxt('auth_answers_count.csv', dtype=int ,delimiter=",")

print("auth_answers_count :")
print(auth_answers_count)

# number_of_survey_taken[i] is the number of survey answered by all_auth_id_array[i]
number_of_survey_taken = np.sum(auth_answers_count, axis=1)
# number_of_participants_by_survey[i] is the number of participants to survey dfs[j]
number_of_participants_by_survey = np.sum(auth_answers_count, axis=0)

print("#######################")
print("number of participant by survey :")
for i in range(4):
    print(dfs[i,0] + " : " + str(number_of_participants_by_survey[i]))

# number_of_participant_to_several_surveys[i] is the number of participants that have
# answerd to i surveys out of the 4 (0<i<5)
number_of_participant_to_several_surveys = np.bincount(number_of_survey_taken)

print("#######################")
print("number of participant to x surveys :")
for i in range(5):
    print(str(number_of_participant_to_several_surveys[i]) + " people have participed to "
          + str(i) + " different surveys.")
    


auth_answers_count :
[[0 0 1 0]
 [1 1 1 0]
 [0 0 0 1]
 ...
 [0 1 0 0]
 [1 0 0 0]
 [0 0 1 0]]
#######################
number of participant by survey :
fiscalite : 53961
democratie : 32424
ecologie : 42656
organisation : 34529
#######################
number of participant to x surveys :
0 people have participed to 0 different surveys.
54322 people have participed to 1 different surveys.
17401 people have participed to 2 different surveys.
9946 people have participed to 3 different surveys.
11152 people have participed to 4 different surveys.


### Resume

Total number of participants by survey (open questions only):

* **fiscalite :** 53961
* **democratie :** 32424
* **ecologie :** 42656
* **organisation :** 34529

Number of participant to $x$ surveys :

* 54322 people have participated to only one survey
* 17401 people have participated to 2 surveys
* 9946 people have participated to 3 survey
* 11152 people have participated to 4 survey

These preliminaries let us think that what we wanted to do for this project is doable, because the number of people who answered on several themes is pretty large. We are now looking for a method to build these type-profiles.


# General ideas about this project

After having processed the data, our project will be articulated in two parts: 

* **Learning :** We are going to build a model able to find different groups (or trends opinion) based on clustering. We will train our model on people who answered the 4 themes. We will make some tests to determine a number of different profiles (maximum of 10 clusters) that seems relevant. Then we will check the relevance of these type profiles.
* **Check :** We will test our model applying it to the test dataset (people who have participated to 3 surveys) to check if it is able to correctly predict the themes.

Based on the result of the last point we will be able to say whether or not our goal is reached, and if the dataset given is enough to achieve this.

Method: the learning will be dividing in 2 stages:

* **1st stage : GMM** We chose to look for 10 clusters based on the answers on each themes, so 40 clusters will be made at the end of this stage.

* **2nd stage : Self-Organizing Map (SOM)** We will try to find only 10 clusters based on the 40 clusters made at the first stage based on a SOM also called Kohonen Map. It is very useful since it is used for dimension reduction and we can easily visualize the final map on a 2-dimensions map.


# First Stage: GMM

We use a **Gaussian Mixture Model** (GMM) because in the clustering method, for each answer of a survey, we can tell the **probability** that it is a part of each cluster. While in other methods, you only have the best cluster. Because of that, the second clustering (SOM) on the top of the first (GMM) will have more **nuanced features**, not only the $4$ clusters in wich this person belongs but the probabilities this person belongs to each of the $40$ clusters.

### Aim of this stage :

As a result, the aim of this stage is to create the matrix $X$ whoses raws coresponds to the the persons who ansered to the open questions of the four surveys. And whoses columns are the probability that this person belongs to each of the $40$ clusters.

### Optimization

To accelerate the calculus we will use the library `multiprocessing` that allows to share the calculus an several CPUs. Moreover, we will separate the extraction of the features of each of the four thems (that are independants).

**This is the code for the preprocessing :**

source files : `preprocessing-democratie.py`, `preprocessing-ecologie.py`, `preprocessing-fiscalite.py` and `preprocessing-organisation.py`

In [ ]:
s = FeaturesExtractor()
def extract_features():
    for k in [0,1,2,3]:
        ids_questions = get_ids_open_reponses(dfs[k,1])
        ids_auth = np.sort(list(set(dfs_responses[k,1]['authorId'].values)))
        responses = (dfs_responses[k,1][:].formattedValue.values.tolist())
        # Extract embeddings for sentences
        features = np.zeros((len(ids_auth), 300*len(ids_questions)))
        for i in range(len(ids_auth)) :
            for j in range(len(ids_questions)) :
                response_unique = dfs_responses[k,1][dfs_responses[k,1]['authorId'] == ids_auth[i]][dfs_responses[k,              1][dfs_responses[k,1]['authorId'] == ids_auth[i]]['questionId'] == ids_questions[j]].formattedValue.values.tolist()
                if (len(response_unique) > 0) :
                    features[i][300*j:300*(j+1)] = s.get_features(response_unique[0])
                else:
                    features[i][300*j:300*(j+1)] = [0.]*300
        np.savetxt(dfs_responses[k,0,]+'_all_questions.tsv', features, delimiter='\t')

Actually, we ran this function in four different processus with $k=0$, $k=1$, $k=2$ and $k=3$.


**This is the code of FeatureExtractor() :**

source file : `kmeans_embeddings.py`

In [ ]:
def tokenize(text):
    return word_tokenize(text, language='french')


class FeaturesExtractor:
    """ Handle features extractions based on word embeddings (fasttext) """
    def __init__(self,
                 model_path: str = 'data/cc.fr.300.bin'):
        assert model_path.endswith('.bin'), 'model_path should be a .bin file'
        assert pathlib.Path(model_path).exists(), 'model_path does not exists'

        self.stop_words = set(stopwords.words('french') +
                              list(string.punctuation) +
                              stop_words.get_stop_words('fr'))

        print(('loading model could take a while...'
               ' and takes up to 7GO of RAM'))
        self.model = fastText.load_model(model_path)
        self.porter = PorterStemmer()

    def get_features(self, response: str):
        """
        """
        assert type(response) == str, 'response must be a string'
        words = tokenize(response)
        # this line deletes the stopwords, keeps alphanumeric only, put the case to lower
        # and stems the words
        words = [self.porter.stem(x.lower()) for x in words if x not in self.stop_words and x.isalpha()]

        return self.model.get_sentence_vector(' '.join(words))

`get_features()` deletes the **stopwords**, keeps **alphanumeric** only, put the case to **lower** and **stems** the words.

After this preprocessing, we have the **features for the GMM**. The next step is to fit and predict the proba for the GMM.

The following code does the job for the organisation theme. The code is adapted to the four themes and run simultanously.

In [ ]:
def fill_X(auth_index):
    global gmm
    global ids_auth
    global features
    global four_surveys_taken_auth_ids
    auth = four_surveys_taken_auth_ids[auth_index]
    k = list(ids_auth).index(auth)
    return gmm.predict_proba(features[k].reshape(1, -1))[0]


n_compo = 10
df_organisation = ut.read_data('data/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.json')
df_resp_org = get_open_reponses(df_organisation)
df_ids_org = get_ids_open_reponses(df_organisation)
four_surveys_taken_auth_ids = np.loadtxt("four_surveys_taken_auth_ids.csv", delimiter=",", dtype=str)
ids_auth = np.sort(list(set(df_resp_org['authorId'].values)))
np.savetxt("ids_auth_sorted.csv", ids_auth, delimiter=",", fmt="%s")
X = np.zeros((len(four_surveys_taken_auth_ids), n_compo))
# read features
features = np.loadtxt('responses organisation_all_questions.tsv', delimiter='\t')
# Fit GMM
gmm = GaussianMixture(n_components=n_compo)
gmm.fit(features)
# pool
local_pool = multiprocessing.Pool(10)
X = np.array(local_pool.map(fill_X, range(len(four_surveys_taken_auth_ids))))
local_pool.close()
local_pool.join()
np.savetxt("X_organisation.csv", X, delimiter=",")

Finally, we concatenate the four $X$ to create the features for the SOM.

# Second Stage: Self-Organizing Map

According to Wikipedia, “A self-organizing map (SOM) or self-organizing feature map (SOFM) is a type of artificial neural network (ANN) that is trained using unsupervised learning to produce a low-dimensional (typically two-dimensional), discretized representation of the input space of the training samples, called a map, and is therefore a method to do dimensionality reduction.”

Here are the most important features of SOM:

* **Dimensionality Reduction:** n dimensional data from input layer will be represented by two-dimensional output layer. It can be also said that SOM can compress data by dimensionality reduction.

* **Competitive Learning:** Unlike most of other ANNs, SOM uses competitive learning which means only one neuron gets activated for a input vector. The winner neuron is called Best Matching Unit (BMU).

* **Topological Preservation:** In SOM, the clusters of similar features are near each other.  In other words, the data points that are nearby in high dimensional space will also be near by in low dimensional representation, hence preserving the topography. This is made possible by neighborhood weight adjustments. When an input vector excites the BMU, the weights of BMU as well as neighboring neurons are adjusted (depending on radius from BMU).

We are going to use the output dataset of the first GMM stage learning: as an input we have for each author_id, a row of 40 colums which represents the proportion of finding itself in one of the 40 clusters. 

**The SOM training goal is to find regular patterns in the 40 clusters in order to make the 10 final clusters.**

It exists two types of SOM training algorithms: unbatched and batched ones. The two training algortithms are implemented but we will only use the batched one since it is adapted to huge volume of datas (big data).

In [17]:
#Let's try to find the best model of SOM: we need to first determine its size
#import som_batched_learning as sbt
import sys
sys.path

['',
 '/Users/maximelhoustau/anaconda3/lib/python36.zip',
 '/Users/maximelhoustau/anaconda3/lib/python3.6',
 '/Users/maximelhoustau/anaconda3/lib/python3.6/lib-dynload',
 '/Users/maximelhoustau/anaconda3/lib/python3.6/site-packages',
 '/Users/maximelhoustau/anaconda3/lib/python3.6/site-packages/aeosa',
 '/Users/maximelhoustau/anaconda3/lib/python3.6/site-packages/IPython/extensions',
 '/Users/maximelhoustau/.ipython']